In [1]:
import time
import random
import threading
from threading import Thread

In Python 3, `threading` is the module used to create and use threads. There's a low level module `_thread` but it's not recommended to use it directly. I'm mentioning it just as a warning, **don't use `_thread`!**.

The most important class in the `threading` module is: `Thread` (doh!).

Very simplified, this is how a thread is instantiated:

```python
class Thread:
    def __init__(self, target, name=None, args=(), kwargs={}):
        pass
```
(there's a `group` argument which should be always `None`, as it's reserved for future use)

In this case, `target` is the function that will be executed in that particular thread.

Once a thread has been _created_ (instantiated), we'll need to `start()` it in order for it to begin to process.

#### Basic example of a thread

In [2]:
def simple_worker():
    print('hello', flush=True)
    time.sleep(2)
    print('world', flush=True)

In [3]:
t1 = Thread(target=simple_worker)

In [4]:
t1.start()

hello
world


In [5]:
t1.join()

#### Running multiple threads in parallel

In [6]:
def simple_worker():
    time.sleep(random.random() * 5)
    value = random.randint(0, 99)
    print(f'My value: {value}')

In [7]:
threads = [Thread(target=simple_worker) for _ in range(5)]

In [8]:
[t.start() for t in threads]

[None, None, None, None, None]

My value: 30
My value: 62
My value: 89
My value: 31
My value: 61


In [9]:
[t.join() for t in threads]

My value: 48
My value: 94
My value: 35
My value: 16
My value: 7


[None, None, None, None, None]

#### Thread States

A thread can be in multiple states, when a thread has just been created, its state is `"ready"`:

In [19]:
def simple_worker():

    print('Thread running...')
    time.sleep(5)
    print('Thread finished...')

In [20]:
t = Thread(target=simple_worker)

In [21]:
t.is_alive()

False

In [22]:
t.start()

Thread running...


In [23]:
t.is_alive()

True

In [24]:
t.join()

Thread finished...


A thread that has finished can't be started again, as shown in the following example:

In [26]:
t.start()

RuntimeError: threads can only be started once

**Important:** It's not possible(\*) to manage thread states manually, for example, stopping a thread. A thread always has to run its natural cycle.

(\*) You might find hacks in the internet on how to stop threads, but **it's a bad practice**. We'll discuss more later.

#### Thread Identity

The thread class has two attributes that lets us identify each thread. The human-ready `name`, which we can set when we construct the thread, and the machine-oriented `ident` one

In [27]:
def simple_worker():
    print('Thread running...')
    time.sleep(5)
    print('Thread exiting...')

In [28]:
t = Thread(target=simple_worker)

In [29]:
t.name

'Thread-15'

`ident` will be `None`until we run the thread.

In [30]:
t.ident is None

True

In [31]:
t.start()

Thread running...


In [32]:
t.name

'Thread-15'

In [33]:
t.ident

7784

Thread exiting...


We can create a thread and assign a custom name to it:

In [34]:
t = Thread(target=simple_worker, name='PyCon 2020 Tutorial!')

In [35]:
t.start()

Thread running...


In [36]:
t.name

'PyCon 2020 Tutorial!'

In [37]:
t.ident

1716

Thread exiting...


#### A thread knows itself

It's also possible to know the identity of the thread from within the thread itself. It might be counter intuitive as we don't have the reference to the created object, but the module function `threading.currentThread()` will provide access to it.

In [38]:
def simple_worker():
    sleep_secs = random.randint(1, 5)
    myself = threading.current_thread()
    ident = threading.get_ident()
    print(f"I am thread {myself.name} (ID {ident}), and I'm sleeping for {sleep_secs}.")
    time.sleep(sleep_secs)
    print(f'Thread {myself.name} exiting...')

In [39]:
t1 = Thread(target=simple_worker, name='Bubbles')
t2 = Thread(target=simple_worker, name='Blossom')
t3 = Thread(target=simple_worker, name='Buttercup')

In [40]:
t1.start()

I am thread Bubbles (ID 1288), and I'm sleeping for 3.


In [41]:
t2.start()

I am thread Blossom (ID 3292), and I'm sleeping for 3.


In [42]:
t3.start()

I am thread Buttercup (ID 11188), and I'm sleeping for 3.
Thread Bubbles exiting...
Thread Blossom exiting...
Thread Buttercup exiting...


In [43]:
print('Waiting...')

Waiting...


#### Passing parameters to threads

Passing parameters is simple with the thread constructor, just use the `args` argument:

In [44]:
def simple_worker(time_to_sleep):
    myself = threading.current_thread()
    ident = threading.get_ident()
    print(f"I am thread {myself.name} (ID {ident}), and I'm sleeping for {time_to_sleep}.")
    time.sleep(time_to_sleep)
    print(f'Thread {myself.name} exiting...')

In [45]:
t1 = Thread(target=simple_worker, name='Bubbles', args=(3, ))
t2 = Thread(target=simple_worker, name='Blossom', args=(1.5, ))
t3 = Thread(target=simple_worker, name='Buttercup', args=(2, ))

In [46]:
t1.start()

I am thread Bubbles (ID 16784), and I'm sleeping for 3.


In [47]:
t2.start()

I am thread Blossom (ID 14604), and I'm sleeping for 1.5.


In [48]:
t3.start()

I am thread Buttercup (ID 7424), and I'm sleeping for 2.
Thread Blossom exiting...
Thread Buttercup exiting...
Thread Bubbles exiting...


#### Subclassing `Thread`

So far, the way we've created threads is by passing a `target` function to be executed. There's an alternative, more OOP-way to do it, which is extending the Thread class:

In [55]:
class MyThread(Thread):
    def __init__(self, time_to_sleep, name=None):
        super().__init__(name=name)
        self.time_to_sleep = time_to_sleep
        
    def run(self):
        ident = threading.get_ident()
        print(f"I am thread {self.name} (ID {ident}), and I'm sleeping for {self.time_to_sleep} secs.")
        time.sleep(self.time_to_sleep)
        print(f'Thread {self.name} exiting...')

In [56]:
t = MyThread(2)

In [57]:
t.start()

I am thread Thread-18 (ID 11864), and I'm sleeping for 2 secs.
Thread Thread-18 exiting...


## Shared Data

As we'll see, Threads can access shared data within the process they live in. Example:

In [58]:
TIME_TO_SLEEP = 2

In [59]:
def simple_worker():
    myself = threading.current_thread()
    print(f"I am thread {myself.name}, and I'm sleeping for {TIME_TO_SLEEP}.")
    time.sleep(TIME_TO_SLEEP)
    print(f'Thread {myself.name} exiting...')

In [60]:
t1 = Thread(target=simple_worker, name='Bubbles')
t2 = Thread(target=simple_worker, name='Blossom')
t3 = Thread(target=simple_worker, name='Buttercup')

In [61]:
t1.start()

I am thread Bubbles, and I'm sleeping for 2.


In [62]:
t2.start()

I am thread Blossom, and I'm sleeping for 2.


In [63]:
t3.start()

I am thread Buttercup, and I'm sleeping for 2.
Thread Bubbles exiting...
Thread Blossom exiting...
Thread Buttercup exiting...


How is this possible?

Remember, all threads live **within the same process**, and the variable `TIME_TO_SLEEP` is stored in the process. So all the threads created can access that variable.

<img src="img/thread_shared_data.png" width=900 />

## A real example

In the `crypto-examples` directory, we've included a real example of a web server that contains prices of different cryptocurrencies. You can run it with `python flask_app.py --sleep [sleep in seconds]`. The server can be slowed down on purpose to simulate a real slow server.

Let's check how to get one price as an example:

In [67]:
BASE_URL = "http://localhost:5000"

In [68]:
import requests

In [69]:
resp = requests.get(f"{BASE_URL}/price/bitfinex/btc/2020-04-01")

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /price/bitfinex/btc/2020-04-01 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026EE2DD1FA0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
resp

In [ ]:
resp.json()

Now, let's suppose we want to get the price of Bitcoin from 3 different exchanges: `bitfinex`, `bitstamp` and `kraken`. The sequential requests would take us 6 seconds (with a sleep param in 2).

In [ ]:
EXCHANGES = ['bitfinex', 'bitstamp', 'kraken']

In [ ]:
start = time.time()

In [ ]:
for exchange in EXCHANGES:
    resp = requests.get(f"{BASE_URL}/price/{exchange}/btc/2020-04-01")
    print(f"{exchange.title()}: ${resp.json()['close']}")

In [ ]:
time.time() - start

Let's now move it to threads! For now, we'll just **print** the output, as we'll se data sharing in further lessons...

In [ ]:
def check_price(exchange, symbol, date, base_url=BASE_URL):
    resp = requests.get(f"{base_url}/price/{exchange}/{symbol}/{date}")
    print(f"{exchange.title()}: ${resp.json()['close']}")

In [ ]:
check_price('bitfinex', 'btc', '2020-04-01')

In [ ]:
threads = [
    Thread(target=check_price, args=(exchange, 'btc', '2020-04-01'))
    for exchange in EXCHANGES
]

In [ ]:
start = time.time()

In [ ]:
[t.start() for t in threads];

In [ ]:
[t.join() for t in threads];

In [ ]:
time.time() - start

#### How many threads can we start?

Let's say we need to get prices for 10 exchanges, 3 symbols, for a total of 30 days. Those are a lot of requests:

In [ ]:
10 * 3 * 30

Can we start 900 threads all at once? Sadly, we can't. Each threads consumes resources and too many threads are usually a problem.

So, what can we do when we need to process too many concurrent jobs? We'll create workers and use a consumer-producer model. But first, we need to talk about shared data, race conditions and synchronization...

## Summary:

* `threading` module ✅
* `_thread`  module ❌

A thread's life cycle is Instantiated > Started > Running > Finished.